## Single-Period Inventory Model

### Example :
* purchase cost = 10
* sales price = 24
* salvage price = 7
* Penalty = 0

### Main Objectives
* 1. Determine optimal value of Q
* 2. Calculate the expected profit

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fitter import Fitter, get_distributions

### Data Cleaning

In [ ]:
all_skus = pd.read_csv('online_retail2.csv')
all_skus = all_skus[all_skus.Quantity > 0]
all_skus = all_skus[all_skus.Price > 0]
all_skus.dropna(inplace= True)
all_skus.drop_duplicates(inplace=True)
# all_skus

### Select the sku and specify time units

In [ ]:
all_skus.StockCode.value_counts()

In [ ]:
sku = all_skus[all_skus.StockCode == '20725']
sku = sku[['Quantity', 'InvoiceDate']].copy().reset_index(drop=True)
# sku

In [ ]:
# sku.InvoiceDate = pd.to_datetime(sku.InvoiceDate)
sku.InvoiceDate = sku.InvoiceDate.astype('datetime64')

In [ ]:
sku['Year'] = sku.InvoiceDate.dt.year.astype('str')
sku['Week'] = sku.InvoiceDate.dt.isocalendar().week.astype('str')
# sku

In [ ]:
sku.Week = [i if len(i)>1 else '0'+i for i in sku.Week]
sku['time_unit'] = sku.Year + '-' + sku.Week
sku.sample(10)

In [ ]:
grp = sku.groupby('time_unit').Quantity.sum().reset_index()
# grp

### Finding and eliminating outliers

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=grp,x='Quantity');
# sns.boxenplot(data=grp,x='Quantity');

In [ ]:
grp = grp[grp.Quantity<800].copy()
plt.figure(figsize=(12,6))
sns.boxplot(data=grp,x='Quantity');

### Find the best fitted distribution

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=grp, bins=50);

In [ ]:
demand_week = grp['Quantity'].values
# demand_week

In [ ]:
plt.figure(figsize=(12,6)) 

plt.figure(figsize=(12,6))
f = Fitter(data=demand_week,
           distributions=['lognorm',
                          'norm',
                          'triang',
                          'gamma',
                          'uniform',
                          'expon',
                          'rayleigh']
           )
f.fit()
f.summary()

In [ ]:
f.get_best(method = 'sumsquare_error')

In [ ]:
from scipy.stats import triang
f.fitted_param['triang']
triang_dist = triang(f.fitted_param['triang'][0],f.fitted_param['triang'][1],f.fitted_param['triang'][2])

### Calculate Q* (Tabular Solution)

In [ ]:
df1 = pd.DataFrame()
df1['Demand'] = np.arange(50, 800, 10)
df1['cumprob'] = triang_dist.cdf(df1.Demand)
df1['prob'] = df1.cumprob.diff().fillna(df1.cumprob)
# df1

In [ ]:
# Parameters
purchase_cost=10
sales_price=24
salvege=7
penalty=0

In [ ]:
df2 = pd.DataFrame()
df2['order_quant'] = np.arange(50,1001,50)

In [ ]:
expected_prof = np.zeros(shape=(len(df2), 1))

for i in range(len(df2)):
    prof_vec = np.zeros(shape=(len(df1), 1))
    q = df2.order_quant[i]

    for j in range(len(df1)):
        d = df1.Demand[j]
        prof_vec[j] = (sales_price * min(d, q) - purchase_cost * q +
                       salvege * max(0, q - d) - penalty * max(0, d - q))
        expected_prof[i] = np.sum(np.array(df1.prob) * np.transpose(prof_vec))

df2['profit'] = expected_prof
# df2

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(data=df2,x='order_quant',y='profit',marker='s', markersize=8);

### Calculate Q* (Analytical Solution)

In [ ]:
net_short_cost = sales_price - purchase_cost +penalty
net_excess_cost = purchase_cost - salvege
CR = net_short_cost /(net_short_cost+net_excess_cost)
CR

In [ ]:
optimal_Q = triang_dist.ppf(q=CR)
optimal_Q

### Calculate Optimal Profit

In [ ]:
q = optimal_Q

def profit_underQ(x):
    return sales_price * x - purchase_cost * q + salvege * (q - x)

def profit_aboveQ(x):
    return sales_price * q - purchase_cost * q - penalty * (x - q)

E_profit_under = triang_dist.expect(func=profit_underQ,lb=0,ub=q)

E_profit_above = triang_dist.expect(func=profit_aboveQ,lb=q)

E_profit = E_profit_under + E_profit_above

E_profit , E_profit_under , E_profit_above

In [ ]:
def profit(x):
    if x < q:
        return sales_price * x - purchase_cost * q + salvege * (q - x)
    else:
        return sales_price * q - purchase_cost * q - penalty * (x - q)


E_profit2 = triang_dist.expect(func=profit)
E_profit2